> Add the tailored models : 

In [ ]:
import os 

path_work = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023"
path_server = f"{path_work}/ensemble_tailored"
path_old_ensemble = f"{path_work}/ensemble_2809"
path_ensemble = f"{path_work}/ensemble_1512"

#tailored_KLtypes = ["KL3" , "KL8", "KL12", "KL13", "KL23", "KL38", "KL46" ,"KL62"]
tailored_KLtypes = ["KL3","KL38"]
# Make sure the rep is created :
try :
    os.mkdir(path_ensemble)
except FileExistsError :
    pass

# Get the regulars there :
for model in os.listdir(path_old_ensemble) : 
    kltype = model.split(".")[0]
    if kltype not in tailored_KLtypes :
        os.system(f"cp {path_old_ensemble}/{model} {path_ensemble}")

# Get the tailored ones :
for model in os.listdir(path_server) : 
    kltype = model.split(".")[0]
    if kltype in tailored_KLtypes :
        os.system(f"cp {path_server}/{model} {path_ensemble}")

***
# Prediction code : 

In [ ]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_1512"



> Make model : 

In [ ]:
dico_tailored = {"KL3" : "small"}
dico_models = TropiGAT_functions.make_ensemble_tailored_TropiGAT(path_ensemble , dico_tailored)

> Ferriol

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"

#Dpo_domains_77.esm2.embedding.csv
dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.1512.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

In [ ]:
dpo_embeddings.index = [x if x.count("__cds") > 0 else x.replace("_cds", "__cds") for x in dpo_embeddings.index]
dpo_embeddings.index = [x.replace("__CDS","__cds") if x.count("__CDS") > 0 else x for x in dpo_embeddings.index]

> Bea

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)


> Towndsend

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)


> Others 

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

***
> run the predictions 

In [ ]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    ferriol_predictions[dpo] = pred

In [ ]:
ferriol_predictions

In [ ]:
# format the results : 
ferriol_pred_formated = TropiGAT_functions.format_predictions(ferriol_predictions , sep = "__cds")
TropiGAT_functions.clean_print(ferriol_pred_formated)

***

In [ ]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred

In [ ]:
# format the results : 
bea_pred_formated = TropiGAT_functions.format_predictions(bea_predictions , sep = "_")
TropiGAT_functions.clean_print(bea_pred_formated)

In [ ]:
towndsend_embeddings.info()

In [ ]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

In [ ]:
# format the results : 
towndsend_pred_formated = TropiGAT_functions.format_predictions(towndsend_predictions , sep = "_")
TropiGAT_functions.clean_print(towndsend_pred_formated)

***

In [ ]:
others_embeddings.info()

In [ ]:
others_embeddings.index

In [ ]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

In [ ]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

In [ ]:
predictions = [other_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.1512.results.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

***
# Write the results : 

In [ ]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.1712.results.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Parse TropiGAT and Seqbased results :

In [ ]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.1712.results.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])
# Seqbased_model.results.bit50.tsv
# Seqbased_model.results.tsv
seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.results.bit50.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])


In [ ]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A_")[0])

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
#merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

In [ ]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]

final_df.protein_id.tolist()

In [ ]:
final_df.to_csv(f"{path_project}/PPT_results.matrices.1512.tsv", sep = "\t", header = True, index = False)